In [3]:
from nltk.parse.stanford import StanfordDependencyParser
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
import spacy
import nltk
from word2number import w2n
from find_similar import *

ps = nltk.PorterStemmer()

/home/guest_aashay/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [4]:
from gensim.models import Word2Vec,KeyedVectors
wvec = KeyedVectors.load_word2vec_format("../btp2/datasets/GoogleNews-vectors-negative300.bin", binary=True)
fl = open("small_verb.txt")
seedverbs = []
from nltk.corpus import wordnet

seedtype = {}
for line in fl:
    seedtype[line.split()[0]] = line.split()[1]
    line = line.strip().split(" ")[0]
    seedverbs.append(line.split()[0])


In [22]:
class linguistic_operations():
    def __init__(self):
        self.depRelations = []
        self.lastrelation = []
        self.lastGraph = []
        self.premGraph = []
        self.question = ""
        self.ind = {}
        self.visited = []
        self.path_to_jar = "./stanford-parser/stanford-parser.jar"
        self.path_to_models_jar = "./stanford-parser/stanford-parser-3.4.1-models.jar"
        self.dependency_parser = StanfordDependencyParser(path_to_jar = self.path_to_jar, path_to_models_jar = self.path_to_models_jar)
        return
    
    
    def dependencyParse(self,sentence):
        self.question = sentence
        result = self.dependency_parser.raw_parse(sentence)
        dep = result.next()
        self.depRelations = list(dep.triples())
        return self.depRelations
    
    
    def numNoun(self):
        allrelations = {}
        lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
        for relation in self.depRelations:
            if relation[1] == 'num':
                seed = ps.stem(relation[0][0])
                if seed not in allrelations:
                    allrelations[seed] = []
                print ("rela = ",str(relation[2][0]))
                allrelations[seed].append(w2n.word_to_num(str(relation[2][0])))
        self.allrelations =  allrelations
        return allrelations
    
    
    def findroot(self):
        for relation in self.lastrelation:
            if relation[0][1][0]=='W':
                return ps.stem(relation[0][0])
            elif relation[2][1][0]=='W':
                return ps.stem(relation[2][0])
        return "NAN"
    
    
    def makeGraphLast(self):
        
        last = nltk.sent_tokenize(self.question)[-1]
        allwords = self.makeseedvocab(last)
        result = self.dependency_parser.raw_parse(last)
        dep = result.next()
        self.lastrelation = list(dep.triples())
        print (self.lastrelation)
        newRelation = []
        for relation in self.lastrelation:
            new = (ps.stem(relation[0][0]),relation[1],ps.stem(relation[2][0]))
            newRelation.append(new)

        
        print (newRelation)
        cnt =0 
        for token in allwords:
            self.ind[token] = cnt
            cnt+=1
    
        graph = [[] for i in range(len(allwords)) ]
        for relation in newRelation:
            graph[self.ind[relation[0]]].append(self.ind[relation[2]])
            graph[self.ind[relation[2]]].append(self.ind[relation[0]])
        self.visited = [0 for i in range(len(allwords))]
        return graph
            
    
    def dfs(self,root,graph):
        self.visited[root] = 1
        for i in range(len(graph[root])):
            if self.visited[graph[root][i]]==0:
                self.dfs(graph[root][i],graph)
        return 
            
        
    
    
    def makeseedvocab(self,last):
        allwords = set()
        sentences = nltk.word_tokenize(last)
        for word in sentences:
            allwords.add(ps.stem(word))
        print (allwords)
        return allwords
    
    
    def whoseQuantity(self):
        for relation in self.lastrelation:
            if (relation[0][1] == "NNS" or relation[0][1] == "NN") and ps.stem(relation[0][0]) in self.allrelations:
                return ps.stem(relation[0][0])
            elif (relation[2][1]=="NNS" or relation[2][1]=="NN") and ps.stem(relation[2][0]) in self.allrelations:
                return ps.stem(relation[2][0])
    
    
            

In [23]:

def calculate(inp_sent):
    
    linguistic = linguistic_operations()
    x = linguistic.dependencyParse(inp_sent)
    allrelation = linguistic.numNoun()
    graph = linguistic.makeGraphLast()
    qentity = linguistic.whoseQuantity()
    trans = findtransition()
    findanswer(allrelation,trans,qentity)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.StanforCoreNLPDependencyParser instead.


[((u'passed', u'VBD'), u'ccomp', (u'have', u'VBP')), ((u'have', u'VBP'), u'nsubj', (u'I', u'PRP')), ((u'have', u'VBP'), u'dobj', (u'books', u'NNS')), ((u'books', u'NNS'), u'num', (u'3', u'CD')), ((u'books', u'NNS'), u'cc', (u'and', u'CC')), ((u'books', u'NNS'), u'conj', (u'pens', u'NNS')), ((u'pens', u'NNS'), u'num', (u'4', u'CD')), ((u'passed', u'VBD'), u'nsubj', (u'I', u'PRP')), ((u'passed', u'VBD'), u'dobj', (u'book', u'NN')), ((u'book', u'NN'), u'num', (u'1', u'CD')), ((u'book', u'NN'), u'amod', (u'more', u'JJR')), ((u'passed', u'VBD'), u'prep', (u'to', u'TO')), ((u'to', u'TO'), u'pobj', (u'tom', u'NNP')), ((u'tom', u'NNP'), u'dep', (u'do', u'VBP')), ((u'do', u'VBP'), u'dep', (u'many', u'JJ')), ((u'many', u'JJ'), u'advmod', (u'How', u'WRB')), ((u'do', u'VBP'), u'nsubj', (u'books', u'NNS')), ((u'do', u'VBP'), u'ccomp', (u'have', u'VBP')), ((u'have', u'VBP'), u'nsubj', (u'I', u'PRP')), ((u'have', u'VBP'), u'advmod', (u'now', u'RB'))]


In [38]:

# print (allrelation)
# print (linguistic.lastrelation)

('rela = ', '3')
('rela = ', '4')
('rela = ', '1')


In [25]:
# print (linguistic.visited)
root = linguistic.findroot()
# linguistic.dfs(linguistic.ind[root],graph)
print (linguistic.visited)
# linguistic.dfs()

[]


set(['do', 'I', 'how', u'book', 'have', u'mani', 'now', '?'])
[((u'have', u'VB'), u'dep', (u'books', u'NNS')), ((u'books', u'NNS'), u'amod', (u'many', u'JJ')), ((u'many', u'JJ'), u'advmod', (u'How', u'WRB')), ((u'have', u'VB'), u'aux', (u'do', u'VBP')), ((u'have', u'VB'), u'nsubj', (u'I', u'PRP')), ((u'have', u'VB'), u'dobj', (u'now', u'RB'))]
[(u'have', u'dep', u'book'), (u'book', u'amod', u'mani'), (u'mani', u'advmod', u'how'), (u'have', u'aux', u'do'), (u'have', u'nsubj', u'I'), (u'have', u'dobj', u'now')]


In [31]:
# type(w2n.word_to_num("3"))
# x = find_similar()


In [36]:
def findtransition():
        sentences = nltk.sent_tokenize(linguistic.question)[0:-1]
        premsentence = ""
        for i in sentences:
            premsentence += i
        result = linguistic.dependency_parser.raw_parse(premsentence)
        dep = result.next()
        deppremise = list(dep.triples())
        for relation in deppremise:
            if relation[0][1][0:2] =="VB":
                return seedtype[x.most_similar(ps.stem(relation[0][0]),seedverbs,wvec)]
            if relation[2][1][0:2] =="VB":
                return seedtype[x.most_similar(ps.stem(relation[2][0]),seedverbs,wvec)]
        return "0"
def findanswer(allrelations,trans,qentity):
    quantities = allrelations[qentity]
    if len(quantities)==1:
        return float(quantities[0])
    ans =0.0
    if trans == '+' or trans =='0':
        for i in quantities:
            ans+=float(i)
    if trans == '-':
        ans = abs(float(quantities[0])-float(quantities[1]))
    return ans
            
    

[(u'passes', 0.7006494402885437), (u'passing', 0.5952332019805908), (u'badly_underthrown', 0.5536887645721436), (u'passed', 0.5499497652053833), (u'dumpoff', 0.535132646560669), (u'passess', 0.5271481275558472), (u'nifty_wraparound', 0.5224149227142334), (u'passs', 0.5210694074630737), (u'eluded_tacklers', 0.49968358874320984), (u'mark_Olukunle', 0.4966002404689789)]


2.0

In [41]:
# x.most_similar("acquire",seedverbs,wvec)

[(u'acquiring', 0.7087350487709045), (u'aquire', 0.6920976638793945), (u'acquired', 0.6788821220397949), (u'acquires', 0.6195156574249268), (u'acquisition', 0.6143543720245361), (u'reacquire', 0.6086896657943726), (u'purchase', 0.601489782333374), (u'Acquire', 0.5971124172210693), (u'Acquiring', 0.5743353962898254), (u'buy', 0.5730166435241699)]


u'get'